# 2.3.2 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought
* Best result: Keine Unterschiede.

### Setup

In [1]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
ava = OpenAI(instructions)
parser = Parser()
runner = Runner()

In [2]:
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F11-01-TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print("Seitenzahl:", len(pdf_prompts))

Seitenzahl: 10


### A) OCR komplettes PDF

In [3]:
instruction = 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
instruction_prompt = parser.text2prompts([instruction])

In [4]:
# A1) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Komplettes PDF auf einmal einlesen. Ziel: Konsistentes Layout.
pdf_text = ava.request([*pdf_prompts, *instruction_prompt])
pprint(pdf_text)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# A2) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten. Ziel: Fokus maximieren und Context-Window klein halten + Seitenzahlen beleiben erhalten.
prompt_chain = list(map(lambda prompt: ava.lambdaRequest([prompt, *instruction_prompt]), pdf_prompts))
msgf11pdf = await asyncio.gather(*prompt_chain)
pprint(msgf11pdf)
%store msgf11pdf

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\begin{center}
\includegraphics[width=0.2\textwidth]{logo.png} \\
\textbf{STADT LAICHINGEN} \\
\textbf{ALB-DONAU-KREIS} \\
\end{center}

\begin{flushright}
19.06.2000 \\
9833 \\
\end{flushright}

\begin{center}
\textbf{SCHRIFTLICHER TEIL (Teil B)} \\
\textbf{BEBAUUNGSPLAN "GEWERBEGEBIET HIMMELREICH"} \\
\textbf{STADT LAICHINGEN, GEMARKUNG FELDSTETTEN, ALB-DONAU-KREIS} \\
\end{center}

Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.

Lageplan M 1: 500

Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:

\begin{itemize}
    \item \textbf{Baugesetzbuch (BauGB)} \\
    in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).
    \item \textbf{Baunutzungsverordnung (BauNVO)} \\
    in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).
    \item \textbf{Planzeichenverordnu

### B) Art der baulichen Nutzung

1. text_prompts
2. img_prompts

In [ ]:
%store -r msgbaunvo_art msgf11pdf
context_art = parser.text2prompts([msgbaunvo_art])

In [ ]:
# B1) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'
results = samantha.similaritySearchWithContext(instruction, msgf11pdf, context_art)
pprint(len(results[1])) # retrieved documents
pprint(results[0]) # message

In [ ]:
# B3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF

instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'
messages = await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_art], img_type="pdf")
pprint(messages)

### C) Maß der baulichen Nutzung

1. text_prompts
2. img_prompts

In [ ]:
%store -r msgbaunvo_maß msgf11pdf
context_maß = parser.text2prompts([msgbaunvo_maß])

In [ ]:
# C1) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".'
results = samantha.similaritySearchWithContext(instruction, msgf11pdf, context_maß)
pprint(len(results[1])) # retrieved documents
pprint(results[0]) # message

In [ ]:
# C3) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF

instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'
messages = await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_maß], img_type="pdf")
pprint(messages)

### D) Bauweise, überbaubare Grundstücksflächen

1. text_prompts
2. img_prompts

In [5]:
%store -r msgbaunvo_bg msgf11pdf
context_bg = parser.text2prompts([msgbaunvo_bg])

In [7]:
# D1) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".'
results = ava.similaritySearchWithContext(instruction, msgf11pdf, context_bg)
pprint(len(results[1])) # retrieved documents
pprint(results[0]) # message

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# D3) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF

instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'
messages = await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_bg], img_type="pdf")
pprint(messages)

### E) Alle Themen in einem Prompt

1. text_prompts (TODO)
2. img_prompts

In [7]:
%store -r msgbaunvo msgf11pdf
context_all = parser.text2prompts([msgbaunvo])

In [ ]:
# E1) Textinformationen: Alle Themen – WITH CONTEXT – IMG_PROMPTS

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
messages = await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_all], img_type="pdf")
pprint(messages)

In [ ]:
# E2) Textinformationen: Alle Themen – WITHOUT CONTEXT – IMG_PROMPTS

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
messages = await ava.extractTextFromImages(instruction, [pdf_path], img_type="pdf")
pprint(messages)